# Supervisor (Polyglot)

In [ ]:
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client.core/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.Core.dll"
#r @"../../../../../../../.nuget/packages/fsharp.json/0.4.1/lib/netstandard2.0/FSharp.Json.dll"

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Threading.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Networking.fs
#!import ../../lib/fsharp/Runtime.fs
#!import ../../lib/fsharp/FileSystem.fs

In [ ]:
open Common
open FileSystem.Operators
open Microsoft.AspNetCore.SignalR.Client

## sendJson

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = Networking.testPortOpen port
    if portOpen then
        try
            let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{port}").Build()
            do! connection.StartAsync () |> Async.AwaitTask
            let! result = connection.InvokeAsync<string>("ClientToServerMsg", json) |> Async.AwaitTask
            do! connection.StopAsync () |> Async.AwaitTask
            trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result.Length: {result |> Option.ofObj |> Option.map String.length}") getLocals
            return Some result
        with ex ->
            trace Critical (fun () -> $"sendJson / port: {port} / json: {json} / ex: {ex |> printException}") getLocals
            return None
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return None
}

## sendObj

In [ ]:
let inline sendObj port obj =
    obj
    |> System.Text.Json.JsonSerializer.Serialize
    |> sendJson port

## awaitCompiler

In [ ]:
type VSCPos = {| line : int; character : int |}
type VSCRange = VSCPos * VSCPos
type RString = VSCRange * string
type TracedError = {| trace : string list; message : string |}
type ClientErrorsRes =
    | FatalError of string
    | TracedError of TracedError
    | PackageErrors of {| uri : string; errors : RString list |}
    | TokenizerErrors of {| uri : string; errors : RString list |}
    | ParserErrors of {| uri : string; errors : RString list |}
    | TypeErrors of {| uri : string; errors : RString list |}

In [ ]:
let inline awaitCompiler port cancellationToken = async {
    let! ct =
        cancellationToken
        |> Option.defaultValue System.Threading.CancellationToken.None
        |> Async.mergeCancellationTokenWithDefaultAsync
    
    let cts = new System.Threading.CancellationTokenSource ()

    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = Networking.getAvailablePort (Some 60) port
        if availablePort <> port then
            inbox.Post port
        else
            let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

            let compilerPath =
                repositoryRoot </> "deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"

            let! exitCode, result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet ""{dllPath}"" port={availablePort}"
                        CancellationToken = Some ct
                        WorkingDirectory = None
                        OnLine = Some <| fun { Line = line } -> async {
                            if line |> String.contains $"Server bound to: http://localhost:{availablePort}" then
                                do! Networking.waitForPortAccess (Some 500) true availablePort |> Async.Ignore

                                let rec loop retry = async {
                                    let getLocals () = $"port: {availablePort} / retry: {retry} / {getLocals ()}"
                                    try
                                        let pingObj = {| Ping = true |}
                                        let! pingResult = pingObj |> sendObj availablePort
                                        trace Verbose (fun () -> $"awaitCompiler / Ping / result: {pingResult}") getLocals
                                    with ex ->
                                        trace Verbose (fun () -> $"awaitCompiler / Ping / ex: {ex |> printException}") getLocals
                                        do! Async.Sleep 10
                                        do! loop (retry + 1)
                                }
                                do! loop 0
                                inbox.Post availablePort
                        }
                    }
            trace Debug (fun () -> $"awaitCompiler / exitCode: {exitCode} / result: {result}") getLocals
            cts.Cancel ()
    }, ct)

    let! serverPort = compiler.Receive ()

    let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{serverPort}").Build ()
    do! connection.StartAsync () |> Async.AwaitTask

    let event = Event<_> ()
    let disposable = connection.On<string> ("ServerToClientMsg", event.Trigger)
    let stream =
        FSharp.Control.AsyncSeq.unfoldAsync
            (fun () -> async {
                let! msg = event.Publish |> Async.AwaitEvent
                return Some (msg |> FSharp.Json.Json.deserialize<ClientErrorsRes>, ())
            })
            ()

    let disposable =
        newDisposable (fun () ->
            disposable.Dispose ()
            connection.StopAsync () |> Async.AwaitTask |> Async.StartImmediate
        )

    return
        serverPort,
        stream,
        cts.Token,
        disposable
}

## getFileUri

In [ ]:
let inline getFileUri path =
    $"file:///{path |> String.trimStart [| '/' |]}"

In [ ]:
let inline getFilePathFromUri uri =
    match System.Uri.TryCreate (uri, System.UriKind.Absolute) with
    | true, uri -> uri.AbsolutePath |> System.IO.Path.GetFullPath
    | _ -> failwith "invalid uri"

In [ ]:
let inline getCompilerPort () =
    13805

## buildFile

In [ ]:
let inline buildFile timeout port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> FileSystem.readAllTextAsync

    let eventFilter = function
        | FileSystem.FileSystemChange.Changed (path, _) when path = $"{fileName}.fsx" -> true
        | _ -> false

    let stream, disposable = fileDir |> FileSystem.watchDirectory eventFilter
    use _ = disposable

    let token, disposable = Threading.newDisposableToken cancellationToken
    use _ = disposable

    let! serverPort, errors, ct, disposable = awaitCompiler port (Some token)
    use _ = disposable

    let fsxContentSeq =
        stream
        |> FSharp.Control.AsyncSeq.choose (function
            | _, (FileSystem.FileSystemChange.Changed (path, Some content) as event)
                when event |> eventFilter
                ->
                Some content
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun content ->
            Some (content |> String.replace "\r\n" "\n"), None
        )

    let inline printErrorData (data : {| uri : string; errors : RString list |}) =
        let fileName = data.uri |> System.IO.Path.GetFileName
        let errors =
            data.errors
            |> List.map snd
            |> String.concat "\n"
        $"{fileName}:\n{errors}"

    let errorsSeq =
        errors
        |> FSharp.Control.AsyncSeq.choose (fun error ->
            match error with
            | FatalError message ->
                Some (message, error)
            | TracedError data ->
                Some (data.message, error)
            | PackageErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | TokenizerErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | ParserErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | TypeErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun (message, error) ->
            None, Some (message, error)
        )
    
    let timerSeq =
        1000
        |> FSharp.Control.AsyncSeq.intervalMs
        |> FSharp.Control.AsyncSeq.map (fun _ -> None, None)

    let outputSeq =
        [ fsxContentSeq; errorsSeq; timerSeq ]
        |> FSharp.Control.AsyncSeq.mergeAll

    let! outputChild =
        ((None, [], 0), outputSeq)
        ||> FSharp.Control.AsyncSeq.scan (
            fun (fsxContentResult, errors, typeErrorCount) (fsxContent, error) ->
                match fsxContent, error with
                | Some fsxContent, None -> Some fsxContent, errors, typeErrorCount
                | None, Some (_, FatalError "File main has a type error somewhere in its path.") ->
                    fsxContentResult, errors, typeErrorCount + 1
                | None, Some error -> fsxContentResult, error :: errors, typeErrorCount
                | None, None when typeErrorCount >= 1 ->
                    fsxContentResult, errors, typeErrorCount + 1
                | _ -> fsxContentResult, errors, typeErrorCount
        )
        |> FSharp.Control.AsyncSeq.takeWhileInclusive (fun (fsxContent, errors, typeErrorCount) ->
            trace Debug (fun () -> $"buildFile / takeWhileInclusive / fsxContent: {fsxContent} / errors: {errors} / typeErrorCount: {typeErrorCount}") getLocals
            match fsxContent, errors with
            | None, [] when typeErrorCount > 2 -> false
            | None, [] -> true
            | _ -> false
        )
        |> FSharp.Control.AsyncSeq.tryLast
        |> Async.withCancellationToken ct
        |> Async.catch
        |> Async.runWithTimeoutAsync timeout
        |> Async.StartChild

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    do! Async.Sleep 30

    let buildFileObj = {| BuildFile = {| uri = fullPath |> getFileUri; backend = "Fsharp" |} |}
    let! _buildFileResult = buildFileObj |> sendObj serverPort

    return!
        outputChild
        |> Async.map (function
            | Some (Ok (Some (message, errors, _))) -> message, errors |> List.distinct |> List.rev
            | Some (Error ex) ->
                trace Critical (fun () -> $"buildFile / error: {ex |> printException}") getLocals
                None, []
            | _ -> None, []
        )
}

## persistCode

In [ ]:
let inline persistCode code = async {
    let tempDir = FileSystem.createTempDirectory ()

    let mainPath = tempDir </> "main.spi"
    do! code |> FileSystem.writeAllTextAsync mainPath

    let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

    let spiprojPath = tempDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {repositoryRoot </> "lib"}
packages:
    |core-
    spiral-
modules:
    main
"""
    do! spiprojCode |> FileSystem.writeAllTextAsync spiprojPath

    let disposable = newDisposable (fun () ->
        ()
        // tempDir |> FileSystem.deleteDirectoryAsync |> Async.Ignore |> Async.RunSynchronously
    )

    return mainPath, disposable
}

## buildCode

In [ ]:
let inline buildCode timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode code
    use _ = disposable
    let port = getCompilerPort ()
    return! mainPath |> buildFile timeout port cancellationToken
}

In [ ]:
//// test

let buildCode timeout cancellationToken code = buildCode timeout cancellationToken code

In [ ]:
//// test

"""inl app () =
    console.write_line "text"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> buildCode 15000 None
|> Async.runWithTimeout 15000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some """let rec closure0 () () : int32 =
    let v0 : string = "text"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""",
        []
    )
)

00:00:00 #1 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:00 #3 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:00 #4 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:00 #5 [Debug] runWithTimeoutAsync / timeout: 500
00:00:00 #6 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:01 #7 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:01 #8 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:01 #9 [Verbose] > Server bound to: http://localhost:13805
00:00:01 #10 [Debug] buildFile / takeWhileInclusive

In [ ]:
//// test

""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> _equal None

00:00:03 #18 [Debug] runWithTimeoutAsync / timeout: 60
00:00:03 #19 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:03 #20 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:03 #21 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:03 #22 [Debug] runWithTimeoutAsync / timeout: 500
00:00:03 #23 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:03 #24 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:03 #25 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:03 #26 [Verbose] > Server bound to: http://localhost:13805
00:00:03 #27 [Debug] buildFile / takeWhile

In [ ]:
//// test

"inl app () =
    0i32

inl a = 1

inl main () =
    app
    |> dyn
    |> ignore
"
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "main.spi:
Global inl/let statements should all return functions known at parse time." ]
    )
)

00:00:13 #45 [Debug] runWithTimeoutAsync / timeout: 60
00:00:13 #46 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:13 #47 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:13 #48 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:14 #51 [Debug] runWithTimeoutAsync / timeout: 500
00:00:14 #52 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:14 #53 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:14 #54 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:14 #55 [Verbose] > Server bound to: http://localhost:13805
00:00:14 #56 [Debug] buildFile / takeWhile

In [ ]:
//// test

"""inl main () =
    1i32 / 0i32
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "An attempt to divide by zero has been detected at compile time." ]
    )
)

00:00:15 #64 [Debug] runWithTimeoutAsync / timeout: 60
00:00:15 #65 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:15 #66 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:15 #67 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:15 #68 [Debug] runWithTimeoutAsync / timeout: 500
00:00:15 #69 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:16 #70 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:16 #71 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:16 #72 [Verbose] > Server bound to: http://localhost:13805
00:00:16 #73 [Debug] buildFile / takeWhile

In [ ]:
//// test

"""inl main () =
    1 + ""
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [
            "main.spi:
Constraint satisfaction error.
Got: string
Fails to satisfy: number"
        ]
    )
)

00:00:17 #83 [Debug] runWithTimeoutAsync / timeout: 60
00:00:17 #84 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:17 #85 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:17 #86 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:18 #88 [Debug] runWithTimeoutAsync / timeout: 500
00:00:18 #89 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:18 #90 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:18 #91 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:18 #92 [Verbose] > Server bound to: http://localhost:13805
00:00:18 #93 [Debug] buildFile / takeWhile

In [ ]:
//// test

"""inl main () =
    x + y
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [
            "main.spi:
Unbound variable.
Unbound variable."
        ]
    )
)

00:00:19 #103 [Debug] runWithTimeoutAsync / timeout: 60
00:00:19 #104 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:20 #105 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:20 #106 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:20 #108 [Debug] runWithTimeoutAsync / timeout: 500
00:00:20 #109 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:20 #110 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:20 #111 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:20 #112 [Verbose] > Server bound to: http://localhost:13805
00:00:20 #113 [Debug] buildFile /

In [ ]:
//// test

"""union a =
    | B
    | c

inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "main.spi:
Expected: uppercase variable" ]
    )
)

00:00:22 #124 [Debug] runWithTimeoutAsync / timeout: 60
00:00:22 #125 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:22 #126 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:22 #127 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:22 #128 [Debug] runWithTimeoutAsync / timeout: 500
00:00:23 #129 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:23 #130 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:23 #131 [Verbose] > Server bound to: http://localhost:13805
00:00:23 #132 [Debug] buildFile / takeWhileInclusive / fsxContent:  / errors: [] / typeErrorCount: 0

In [ ]:
//// test

"""
/// abc
inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "main.spi:
Expected: whitespace" ]
    )
)

00:00:24 #140 [Debug] runWithTimeoutAsync / timeout: 60
00:00:24 #141 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:24 #143 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:24 #144 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:24 #145 [Debug] runWithTimeoutAsync / timeout: 500
00:00:24 #146 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:24 #147 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:24 #148 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:24 #149 [Verbose] > Server bound to: http://localhost:13805
00:00:24 #150 [Debug] buildFile /

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox ()
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "Cannot apply a forall with a term." ]
    )
)

00:00:25 #157 [Debug] runWithTimeoutAsync / timeout: 60
00:00:25 #158 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:25 #159 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:25 #160 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:26 #162 [Debug] runWithTimeoutAsync / timeout: 500
00:00:26 #163 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:26 #164 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:26 #165 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:26 #166 [Verbose] > Server bound to: http://localhost:13805
00:00:26 #167 [Debug] buildFile /

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox `i32 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "The main function should not have a forall." ]
    )
)

00:00:28 #177 [Debug] runWithTimeoutAsync / timeout: 60
00:00:28 #178 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:28 #179 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:28 #180 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:28 #181 [Debug] runWithTimeoutAsync / timeout: 500
00:00:28 #182 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:29 #183 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:29 #184 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:29 #185 [Verbose] > Server bound to: http://localhost:13805
00:00:29 #186 [Debug] buildFile /

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.1 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "0.3325000000000001\n",
        []
    )
)

00:00:30 #196 [Debug] runWithTimeoutAsync / timeout: 60
00:00:30 #197 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:30 #198 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:30 #199 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:31 #201 [Debug] runWithTimeoutAsync / timeout: 500
00:00:31 #202 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:31 #203 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:31 #204 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:31 #205 [Verbose] > Server bound to: http://localhost:13805
00:00:31 #206 [Debug] buildFile /

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.01 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
// |> _equal None
// |> fun x -> printfn $"{x.ToDisplayString ()}"

Some((, [])) Value (, []) Item1 <null> Item2 [ ]

00:00:33 #216 [Debug] runWithTimeoutAsync / timeout: 60
00:00:33 #217 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-983> }
00:00:33 #218 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:33 #219 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:33 #220 [Debug] runWithTimeoutAsync / timeout: 500
00:00:33 #221 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:34 #222 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:34 #223 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:34 #224 [Verbose] > Server bound to: http://localhost:13805
00:00:34 #225 [Debug] buildFile /

## getFileTokenRange

In [ ]:
let inline getFileTokenRange port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> FileSystem.readAllTextAsync
    let lines = code |> String.split [| '\n' |]

    let token, disposable = Threading.newDisposableToken cancellationToken
    use _ = disposable

    let! serverPort, _errors, ct, disposable = awaitCompiler port (Some token)
    use _ = disposable

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    let fileTokenRangeObj =
        {|
            FileTokenRange =
                {|
                    uri = fullPath |> getFileUri
                    range =
                        [|
                            {| line = 0; character = 0 |}
                            {| line = lines.Length - 1; character = lines.[lines.Length - 1].Length |}
                        |]
                |}
        |}
    let! fileTokenRangeResult =
        fileTokenRangeObj
        |> sendObj serverPort
        |> Async.withCancellationToken ct

    return fileTokenRangeResult |> Option.map FSharp.Json.Json.deserialize<int array>
}

## getCodeTokenRange

In [ ]:
let inline getCodeTokenRange cancellationToken code = async {
    let! mainPath, disposable = persistCode code
    use _ = disposable
    let port = getCompilerPort ()
    return! mainPath |> getFileTokenRange port cancellationToken
}

In [ ]:
//// test

"""inl main () = ()"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _equal (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 8; 0; 0; 1; 1; 8; 0 |])

00:00:52 #594 [Debug] runWithTimeoutAsync / timeout: 60
00:00:52 #595 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-434> }
00:00:52 #596 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:52 #597 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:52 #599 [Debug] runWithTimeoutAsync / timeout: 500
00:00:53 #600 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:53 #601 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:53 #602 [Verbose] > Server bound to: http://localhost:13805
00:00:53 #603 [Debug] sendJson / port: 13805 / json: {"FileOpen":{"spiText":"inl main () = ()","uri":"file:

In [ ]:
//// test

"""inl main () = 1i32"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _equal (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 3; 0; 0; 1; 3; 12; 0 |])

00:00:57 #609 [Debug] runWithTimeoutAsync / timeout: 60
00:00:57 #610 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-801> }
00:00:57 #611 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:57 #612 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:58 #614 [Debug] runWithTimeoutAsync / timeout: 500
00:00:58 #615 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:58 #616 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:58 #617 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:58 #618 [Verbose] > Server bound to: http://localhost:13805
00:00:58 #619 [Debug] sendJson / port: 1

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | BuildFile of string * string
    | FileTokenRange of string * string
    | ExecuteCommand of string
    | Timeout of int
    | Port of int

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | BuildFile _ -> nameof BuildFile
            | FileTokenRange _ -> nameof FileTokenRange
            | ExecuteCommand _ -> nameof ExecuteCommand
            | Timeout _ -> nameof Timeout
            | Port _ -> nameof Port

## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let buildFileActions =
        argsMap
        |> Map.tryFind (nameof Arguments.BuildFile)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.BuildFile (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let fileTokenRangeActions =
        argsMap
        |> Map.tryFind (nameof Arguments.FileTokenRange)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.FileTokenRange (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let executeCommandActions =
        argsMap
        |> Map.tryFind (nameof Arguments.ExecuteCommand)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.ExecuteCommand command -> Some command
            | _ -> None
        )

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 60000 * 60

    let port =
        match argsMap |> Map.tryFind (nameof Arguments.Port) with
        | Some [ Arguments.Port port ] -> Some port
        | _ -> None

    async {
        let! buildFileResult =
            buildFileActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let port = port |> Option.defaultWith getCompilerPort
                let! outputCode, errors = inputPath |> buildFile timeout port None
                
                errors
                |> List.map snd
                |> List.iter (fun error ->
                    trace Critical (fun () -> $"main / error: {error}") getLocals
                )

                match outputCode with
                | Some outputCode ->
                    do! outputCode |> FileSystem.writeAllTextAsync outputPath
                    return 0
                | None ->
                    return 1
            })
            |> Async.Sequential

        let! fileTokenRangeResult =
            fileTokenRangeActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let port = port |> Option.defaultWith getCompilerPort
                let! tokenRange = inputPath |> getFileTokenRange port None
                match tokenRange with
                | Some tokenRange ->
                    do! tokenRange |> FSharp.Json.Json.serialize |> FileSystem.writeAllTextAsync outputPath
                    return 0
                | None ->
                    return 1
            })
            |> Async.Sequential
        
        let! executeCommandResult =
            executeCommandActions
            |> List.map (fun command -> async {
                let port = port |> Option.defaultWith getCompilerPort

                let localToken, disposable = Threading.newDisposableToken None
                use _ = disposable

                let! serverPort, _errors, compilerToken, disposable = awaitCompiler port (Some localToken)
                use _ = disposable
                
                let! exitCode, result =
                    Runtime.executeWithOptionsAsync
                        {
                            Command = command
                            CancellationToken = Some compilerToken
                            WorkingDirectory = None
                            OnLine = None
                        }

                trace Debug (fun () -> $"main / executeCommand / exitCode: {exitCode}") getLocals

                return exitCode
            })
            |> Async.Sequential
            
        return
            [| buildFileResult; fileTokenRangeResult; executeCommandResult |]
            |> Array.collect id
            |> Array.sum
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0